# BLU02 - Learning Notebook - Data wrangling workflows - Part 2 of 3

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd
import os

# 2 Combining dataframes in Pandas

## 2.1 How many programs are there per season?

How many different programs does the NYP typically present per season?

Programs are under `/data/programs/` which contains a file per Season.

### Concatenate

To analyze how many programs there are per season, over time, we need a single dataframe containing *all* seasons.

Concatenation means, in short, to unite multiple dataframes (or series) in one. 

The `pd.concat()` function performs concatenation operations along an axis (`axis=0` for index and `axis=1` for columns).

In [ ]:
season_0 = pd.read_csv('./data/programs/1842-43.csv')
season_1 = pd.read_csv('./data/programs/1843-44.csv')

seasons = [season_0, season_1]
pd.concat(seasons, axis=1)

Concatenating like this makes no sense, as we no longer have a single observation per row.

What we want to do instead is to concatenate the dataframe along the index.

In [ ]:
pd.concat(seasons, axis=0)

This dataframe looks better, but there's something weird with the index: it's not unique anymore.

Different observations share the same index. Not cool.

For dataframes that don't have a meaningful index, you may wish to ignore the indexes altogether.

In [ ]:
pd.concat(seasons, axis=0, ignore_index=True)

Now, let's try something different. 

Let's try to change the name of the columns, so that each dataframe has different ones, before concatenating.

In [ ]:
season_0_ = season_0.copy()
season_0_.columns = [0, 1, 2, 'Season']
seasons_ = [season_0_, season_1]
pd.concat(seasons_, axis=0)

What a mess! What did we learn?

* When the dataframes have different columns, `pd.concat()` will take the union of all dataframes by default (no information loss)
* Concatenation will fill columns that are not present for specific dataframes with `np.NaN` (missing values).

The good news is that you can set how you want to glue the dataframes in regards to the other axis, the one not being concatenated. 

Setting `join='inner'` will take the intersection, i.e., the columns that are present in all dataframes.

In [ ]:
pd.concat(seasons_, axis=0, join='inner')

There you go. Concatenation complete.

### Append

The method `df.append()` is a shortcut for `pd.concat()`, that can be called on either a `pd.DataFrame` or a `pd.Series`.

In [ ]:
season_0.append(season_1)

It can take multiple objects to concatenate as well. Please note the `ignore_index=True`.

In [ ]:
season_2 = pd.read_csv('./data/programs/1844-45.csv')

more_seasons = [season_1, season_2]
season_0.append(more_seasons, ignore_index=True)

We are good to go. Let's use `pd.concat` to combine all seasons into a great dataframe.

In [ ]:
def read_season(file):
    path = os.path.join('.', 'data', 'programs', file)
    return pd.read_csv(path)

files = os.listdir('./data/programs/')
files = [f for f in files if '.csv' in f]

A logical approach would be to iterate over all files and appending all of them to a single dataframe.

In [ ]:
%%timeit

programs = pd.DataFrame()
for file in files:
    season = read_season(file)
    programs = programs.append(season, ignore_index=True)

It is worth noting that both `pd.concat()` and `df.append()` make a full copy of the data and continually reusing this function can create a significant performance hit. 

Instead, use a list comprehension if you need to use the operation several times. 

This way, you only call `pd.concat()` or `df.append()` once.

In [ ]:
%%timeit

seasons = [read_season(f) for f in files if '.csv' in f]
programs = pd.concat(seasons, axis=0, ignore_index=True)

In [ ]:
seasons = [read_season(f) for f in files if '.csv' in f]
programs = pd.concat(seasons, axis=0, ignore_index=True)

Now that we have the final `programs` dataframe, we can see how the number of distinct programs changes over time.

In [ ]:
programs['Season'] = pd.to_datetime(programs['Season'].str[:4])

(programs.groupby('Season')
         .size()
         .plot(legend=False, use_index=True, figsize=(10, 7),
               title='Number of programs per season (from 1842-43 to 2016-17)'));

The NYP appears to be investing in increasing the number of distinct programs per season since '95. 

## 2.2 How many concerts are there per season?

What about the number of concerts? The first thing we need to do is to import the `concerts.csv` data.

In [ ]:
concerts = pd.read_csv('./data/concerts.csv')
concerts.head()

We will use the Leon Levy Digital Archives ID (`GUID`) to identify each program.

Now, we have information regarding all the concerts that took place and the season for each program.

The problem? Information about the concert and the season are in different tables, and the program is the glue between the two. Familiar?

### Merge

Pandas provides high-performance join operations, very similar to SQL.

The method `df.merge()` method provides an interface for all database-like join methods.

In [ ]:
?pd.merge

We can call `pd.merge` to join both tables on the `GUID` (and the `ProgramID`, that provides similar info).

In [ ]:
# Since GUID and ProgramID offer similar info, we will drop the later.
programs = programs.drop(columns='ProgramID')

df = pd.merge(programs, concerts, on='GUID')
df.head()

Or, alternatively, we can call `merge()` directly on the dataframe.

In [ ]:
df_ = programs.merge(concerts, on='GUID')
df_.head()

The critical parameter here is the `how`. Since we are not explicitly using it, the merge default to `inner` (for inner-join) by default.

But, in fact, you can use any join, just like you did in SQL: `left`, `right`, `outer` and `inner`.

Remember?

![](../media/types_of_joins.jpg)

*Fig. 1 - Types of joins in SQL, note how left, right, outer and inner translate directly to Pandas.*

A refresher on different types of joins, all supported by Pandas:

| Pandas                                         | SQL              | What it does                              |
| ---------------------------------------------- | ---------------- | ----------------------------------------- |
| `pd.merge(right, left, on='key', how='left')`  | LEFT OUTER JOIN  | Use all keys from left frame only         |
| `pd.merge(right, left, on='key', how='right')` | RIGHT OUTER JOIN | Use all keys from right frame only        |
| `pd.merge(right, left, on='key', how='outer')` | FULL OUTER JOIN  | Use union of keys from both frames        |
| `pd.merge(right, left, on='key', how='inner')` | INNER JOIN       | Use intersection of keys from both frames |

In this particular case, we have:
* A one-to-many relationship (i.e., one program to many concerts)
* Since every single show in `concerts` has a match in `programs`, the type of join we use doesn't matter.

We can use the `validate` argument to automatically check whether there are unexpected duplicates in the merge keys and check their uniqueness.

In [ ]:
df__ = pd.merge(programs, concerts, on='GUID', how='outer', validate="one_to_many")
assert(concerts.shape[0] == df_.shape[0] == df__.shape[0])

Back to our question, how is the number of concerts per season evolving?

In [ ]:
(programs.merge(concerts, on='GUID')
         .groupby('Season')
         .size()
         .plot(legend=False, use_index=True, figsize=(10, 7),
               title='Number of concerts per season (from 1842-43 to 2016-17)'));

Likewise, the number of concerts seems to be trending upwards since about 1995, which could be a sign of growing interest in the genre.

### Join

Now, we want the top-3 composer in total appearances.

Without surprise, we start by importing `works.csv`.

In [ ]:
works = pd.read_csv('./data/works.csv',index_col='GUID')

Alternatively, we can use `df.join()` instead of `df.merge()`. 

There are, however, differences in the default behavior: for example `df.join` uses `how='left'` by default.

Let's try to perform the merge.

In [ ]:
(programs.merge(works, on="GUID")
         .head(n=3))

In [ ]:
programs.merge(works, on="GUID").shape

In [ ]:
(programs.join(works, on='GUID')
         .head(n=3))

# equivalent to
# pd.merge(programs, works, left_on='GUID', right_index=True,
#          how='left').head(n=3)

In [ ]:
programs.join(works, on="GUID").shape

We noticed that the shape of the results is diferent, we have a different number of lines in each one of the methods.
Typically, you would use `df.join()` when you want to do a left join or when you want to join on the index of the dataframe on the right.

Now for our goal: what are the top-3 composers?

In [ ]:
(programs.join(works, on='GUID')
         .groupby('ComposerName')
         .size()
         .nlargest(n=3))

Wagner wins!

What about the top-3 works?

In [ ]:
(programs.join(works, on='GUID')
         .groupby(['ComposerName', 'WorkTitle'])
         .size()
         .nlargest(n=3))

Wagner wins three times!